# Enriching Knowledge Graphs using Language Models

This notebook is part of the experiments done in the paper *Enhancing the semantics of a knowledge graph using language models*. Thus, in this notebook, we apply language models like BERT, RoBERTa and GPT-2 for enriching entities and predicting new relations between these entities.

## 1. Installs and data download

For the models we will use the transformers library while for the experimentation we will use Wikidata5m as our main dataset. Wikidata5m is a million-scale knowledge graph dataset with aligned corpus that integrates the Wikidata knowledge graph and Wikipedia pages.

* https://huggingface.co/docs/transformers/main/en/index
* https://deepgraphlearning.github.io/project/wikidata5m

Transformers library installation: 

In [2]:
# !pip3 install transformers

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.4 MB 5.1 MB/s            
     |████████████████████████████████| 3.6 MB 14.2 MB/s            
     |████████████████████████████████| 192 kB 51.1 MB/s            
     |████████████████████████████████| 101 kB 14.9 MB/s           
     |████████████████████████████████| 289 kB 16.4 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Data download:

In [18]:
# !wget -O wikidata5m_text.txt.gz https://www.dropbox.com/s/7jp4ib8zo3i6m10/wikidata5m_text.txt.gz?dl=1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: no matches found: https://www.dropbox.com/s/7jp4ib8zo3i6m10/wikidata5m_text.txt.gz?dl=1


In [10]:
# !wget -O wikidata5m_transductive.tar.gz https://www.dropbox.com/s/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz?dl=1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: no matches found: https://www.dropbox.com/s/6sbhm0rwo4l73jq/wikidata5m_transductive.tar.gz?dl=1


In [ ]:
# !wget -O wikidata5m_alias.tar.gz https://www.dropbox.com/s/lnbhc8yuhit4wm5/wikidata5m_alias.tar.gz?dl=1

## 2. Functions and necessary imports

In this section we import all the necessary libraries. In addition, we create two functions that we will use later. One to save JSON files and another to count the number of tokens in a sentence according to the GPT-2 tokeniser.

In [20]:
import transformers
from transformers import pipeline
import gzip
import tarfile
import json
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def save_json(data, file_name='results.json'):
    with open(file_name, 'w') as outfile:
        json.dump(data, outfile)
        
def get_n_tokens(string):
    return len(tokenizer(string)['input_ids'])

## 3. Extract data

Unzip the downloaded packages

In [29]:
# tar = tarfile.open("wikidata5m_transductive.tar.gz")
# tar.extractall()
# tar.close()


In [5]:
# tar = tarfile.open("wikidata5m_alias.tar.gz")
# tar.extractall()
# tar.close()
      

We extract the triples (entity 1, relation and entity 2). Note that we are using the validation set due to the big size of the data.

In [4]:
triples = []
with open("wikidata5m_transductive_valid.txt") as f:
    for line in f:
        triples.append(line[:-1].split('\t'))

We create a dictionary with all the relations and their aliases. In this way, each key in the dictionary will be the id for the relation and its value is a list with all the different aliases of that relation.

In [6]:
relations = {}
with open("wikidata5m_relation.txt") as f:
    for line in f:
        l = line[:-1].split('\t')
        key = l[0]
        rest = l[1:]
        relations[key] = rest 

In the same way, we create a dictionary with all the entities and their aliases. In this way, each key in the dictionary will be the id for the entity and its value is a list with all the different aliases of that entity.

In [7]:
entities = {}
with open("wikidata5m_entity.txt") as f:
    for line in f:
        l = line[:-1].split('\t')
        key = l[0]
        rest = l[1:]
        entities[key] = rest 

We create a dictionary (entities_labels) for the entity enrichment with all labels. In this way, the key will be the entity identifier and its value all the different aliases together.

In [10]:
entities_labels={}
for key,value in entities.items():
    entities_labels[key] = (' ').join(value) 

We create a dictionary (entities_context_description) for the entity enrichment with its context. In this way, the key will be the entity identifier and its value the description of that entity.

In [8]:
entity_lookup = {}
with gzip.open('wikidata5m_text.txt.gz', 'rt') as f:
    for line in f:
        a = line.split('\t')
        entity_lookup[a[0]] = a[1]

In [29]:
entities_context_description={}
        
for key,value in entity_lookup.items():
    if key in entities_labels.keys():
        entities_context_description[key] = entities_labels[key] + value 

## 4. Prepare GPT-2 and mask language models

We load the language models that we will use in the relations prediction as well as the entity enrichment.

We load GPT-2 model for text generation in order to enrich the entities.

In [2]:
text_generator = pipeline("text-generation", max_new_tokens=2)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
2022-07-07 12:18:34.366025: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-07 12:18:36.635940: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


We load roberta for prediction of relations between entities with its ability for masking.

In [3]:
fill_mask = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


We load BERT for prediction of relations between entities with its ability for masking.

In [67]:
fill_mask_bert = pipeline('fill-mask', model='bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


# Basic model

These are the experiments done without any kind of entity enrichments. In this way we have three different experiments:

1. Using GPT2 for predicting the 2nd entity giving the first entity and its relation.
2. Using Roberta mask for predicting the relation between two entities.
3. Using BERT mask for predicting the relation between two entities.

## GPT-2 for 2nd entity

This experiment predicted the 2nd entity giving to a GPT-2 model the first entity and the relation. Finally, the results are saved in a JSON file.

In [87]:
results = []

for i,t in enumerate(triples[:3]):
    print(i,'/',len(triples))
    e1,r,e2 = t
    length = len((entities[e1][0] + relations[r][0]).split(' ')) + 4
    res = text_generator(entities[e1][0] + relations[r][0], max_length = length+2, do_sample=False)
    
    entity_prediction_length = len(res[0]["generated_text"]) - len(entities[e1][0] + relations[r][0]) - 1
    entity_prediction = res[0]["generated_text"][-entity_prediction_length:]

    x = {'e1': entities[e1][0], 'r': relations[r][0], 'original_e2':entities[e2][0], 'predicted_e2': entity_prediction}
    results.append(x)
    
save_json(results, file_name='e2_prediction_basic_gpt2_2tokens.json')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0 / 5163


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


1 / 5163


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


2 / 5163


## Masks for relation

The results of the previous experiments were not good. This is because GPT2 is good for generating text but is not good in the context (i.e. it will create a sentence that makes sense but that is not true). This is why we decided to focus on the relations.

### Roberta

This experiment predicted the relation giving to a Roberta mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we do not enrich any entity.

In [66]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities.keys() and e2 in entities.keys():
        masks[i] = entities[e1][0] + ' <mask> ' + entities[e2][0]

predictions = fill_mask(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities[e1][0], relations[original_r][0], entities[e2][0]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_basic_roberta.json')

### BERT

This experiment predicted the relation giving to a BERT mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we do not enrich any entity.

In [69]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities.keys() and e2 in entities.keys():
        masks[i] = entities[e1][0] + ' [MASK] ' + entities[e2][0]

predictions = fill_mask_bert(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities[e1][0], relations[original_r][0], entities[e2][0]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_basic_BERT.json')

# Models with entities enrichment (labels)

These are the experiments done with alias entity enrichment. This mean that for each entity, instead of using the most common alias, we will use all its aliases together (e.g. *Barack Obama* would be *Barack Obama Barack Hussein Obama II Barack Obama II...*). The reason for choosing this enrichment is that it has been proof that when more context is added, the language models improve their performance. In this way we have two different experiments:

1. Using Roberta mask for predicting the relation between two entities.
2. Using BERT mask for predicting the relation between two entities.

## Roberta

This experiment predicted the relation giving to a Roberta mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use alias entinty enrichment for the first and second entity.

In [26]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities_labels.keys() and e2 in entities_labels.keys():
        masks[i] = entities_labels[e1][-252:] + ' <mask> ' + entities_labels[e2][:252]

predictions = fill_mask(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities_labels[e1], relations[original_r][0], entities_labels[e2]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_labels_roberta.json')

## BERT

This experiment predicted the relation giving to a BERT mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use alias entinty enrichment for the first and second entity.

In [107]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities_labels.keys() and e2 in entities_labels.keys():
        masks[i] = entities_labels[e1][-252:] + ' [MASK] ' + entities_labels[e2][:252]

predictions = fill_mask_bert(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities_labels[e1], relations[original_r][0], entities_labels[e2]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_labels_BERT.json')

# Models with entities enrichment (definition)


These are the experiments done with context entity enrichment. This mean that for each entity, instead of using the most common alias, we will use its alias with the description of that entity (e.g. *Barack Obama* would be *Barack Obama is an American politician who served as the 44th president of the United States...*). The reason for choosing this enrichment is that it has been proof that when more context is added, the language models improve their performance. In this way we have two different experiments:

1. Using Roberta mask for predicting the relation between two entities.
2. Using BERT mask for predicting the relation between two entities.

## Roberta

This experiment predicted the relation giving to a Roberta mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use context entinty enrichment for the first and second entity.

In [30]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities_context_description.keys() and e2 in entities_context_description.keys():
        masks[i] = entities_context_description[e1][-252:] + ' <mask> ' + entities_context_description[e2][:252]

predictions = fill_mask(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities_labels[e1], relations[original_r][0], entities_labels[e2]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_context_roberta.json')

## BERT

This experiment predicted the relation giving to a BERT mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use context entinty enrichment for the first and second entity.

In [31]:
results = []
masks = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities_context_description.keys() and e2 in entities_context_description.keys():
        masks[i] = entities_context_description[e1][-252:] + ' [MASK] ' + entities_context_description[e2][:252]

predictions = fill_mask_bert(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = entities_labels[e1], relations[original_r][0], entities_labels[e2]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_context_BERT.json')

# Models with entities enrichment (MASK + GPT-2)


These are the experiments done with GPT-2 entity enrichment. This mean that for the first entity, instead of using the most common alias, we will use its alias with two words generated by GPT-2 (e.g. *Barack Obama* would be *Barack Obama is an*). The reason for choosing this enrichment is that as the mask can only predict one word, we saw that most of the predictions were connectors (e.g. and, or...) so we use GPT-2 with the idea of generating those connectors. In this way we have two different experiments:

1. Using Roberta mask for predicting the relation between two entities.
2. Using BERT mask for predicting the relation between two entities.

## Roberta

This experiment predicted the relation giving to a Roberta mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use GPT-2 entinty enrichment for the first entity.

In [27]:
results = []
masks = {}
gpt2_results = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities.keys() and e2 in entities.keys():
        n_tokens = get_n_tokens(entities[e1][0]) + 2
        res = text_generator(entities[e1][0], max_length = n_tokens, do_sample=False)
        
        gpt2_results[i] = res[0]["generated_text"]
        masks[i] = res[0]["generated_text"] + ' <mask> ' + entities[e2][0]

predictions = fill_mask(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = gpt2_results[k], relations[original_r][0], entities[e2][0]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_maskgpt_roberta.json')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

## BERT

This experiment predicted the relation giving to a BERT mask model the first entity and the second entity. Finally, the results are saved in a JSON file. Note that in this experiment we use GPT-2 entinty enrichment for the first entity.

In [28]:
results = []
masks = {}
gpt2_results = {}

for i,t in enumerate(triples):
    e1,r,e2 = t
    if e1 in entities.keys() and e2 in entities.keys():
        n_tokens = get_n_tokens(entities[e1][0]) + 2
        res = text_generator(entities[e1][0], max_length = n_tokens, do_sample=False)
        
        gpt2_results[i] = res[0]["generated_text"]
        masks[i] = res[0]["generated_text"] + ' [MASK] ' + entities[e2][0]

predictions = fill_mask_bert(list(masks.values()))

i = 0
for k,v in masks.items():
    e1,original_r,e2 = triples[k]
    e1,original_r,e2 = gpt2_results[k], relations[original_r][0], entities[e2][0]
    p = predictions[i]
    x = {'e1': e1, 'e2': e2, 'original_r': original_r, 'predicted_r': p[0]['token_str']}
    results.append(x)
    
    i+=1

save_json(results, file_name='r_prediction_maskgpt_bert.json')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) t